# Measurement Grouping

Since current quantum hardware is limited to single-qubit projective measurement, only terms commuting within individual qubit's subspace can be measured together. These terms are said to be qubit-wise commuting (QWC). 

Thus, one can not measure the entire electronic Hamiltonian $\hat H$ at once, and instead needs to separate it into fragments. 
$$\hat H = \sum_n \hat H_n$$
where each $\hat H_n$ is a QWC fragment. 

In [41]:
from utility import *
import tequila as tq
import pickle as pkl

Here we use $H_2$ as an example for finding QWC fragments. Notice below that each fragment has the same terms on all qubits.  

To show differences between QWC and more advanced grouping, we didn't use the qubit-tappering techinique shown in step 2.

In [4]:
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf='jw')

qwc_list = get_qwc_group(h2)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
0.13716572937099494 [Z0] +
0.15660062488237958 [Z0 Z1] +
0.10622904490856085 [Z0 Z2] +
0.15542669077992843 [Z0 Z3] +
0.13716572937099494 [Z1] +
0.15542669077992843 [Z1 Z2] +
0.10622904490856085 [Z1 Z3] +
-0.1303629205710914 [Z2] +
0.1632676867356435 [Z2 Z3] +
-0.13036292057109133 [Z3]

Fragments 2:
-0.04919764587136759 [X0 X1 Y2 Y3]

Number of fragments: 5


By applying extra unitaries, one may rotate more terms of $\hat H$ into a QWC fragment.  

Recall that in digital quantum computing, the expectation value of $\hat H_n$ given a trial wavefunction $|\psi\rangle$ is 
$$ E_n =\ \langle\psi| \hat H_n | \psi\rangle$$
Inserting unitary transformation $\hat U_n$ does not change the expectation value.
$$ E_n =\ \langle\psi| \hat U_n^\dagger \hat U_n \hat H_n \hat U_n^\dagger \hat U_n  |\psi\rangle$$ 
This nonetheless changes the trial wavefunction and the terms to be measured. 
$$ |\psi\rangle \rightarrow \hat U_n |\psi\rangle = |\phi\rangle$$
$$ \hat H_n \rightarrow \hat U_n \hat H_n \hat U_n^\dagger = \hat A_n$$
The transformation of $|\psi \rangle$ can be done on the quantum computer, and the transformation of $\hat H_n$ is possible on the classical computer. 

Now, although $\hat A_n$ needs to be a QWC fragment to be measurable on a quantum computer, $\hat H_n$ does not. 
Instead, if we restrict $\hat U_n$ to be a clifford operation, the terms in $\hat H$ need only mutually commute. 

Here, we obtain measurable parts of $H_2$ by partitioning its terms into mutually commuting fragments. 

In [25]:
comm_groups = get_commuting_group(h2)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

Number of mutually commuting fragments: 2
The first commuting group
-0.3276081896748089 [] +
-0.04919764587136759 [X0 X1 Y2 Y3] +
0.04919764587136759 [X0 Y1 Y2 X3] +
0.04919764587136759 [Y0 X1 X2 Y3] +
-0.04919764587136759 [Y0 Y1 X2 X3] +
0.15660062488237958 [Z0 Z1] +
0.10622904490856085 [Z0 Z2] +
0.15542669077992843 [Z0 Z3] +
0.15542669077992843 [Z1 Z2] +
0.10622904490856085 [Z1 Z3] +
0.1632676867356435 [Z2 Z3]


To see this fragment is indeed measurable, one can construct the corresponding unitary operator $\hat U_n$.

In [11]:
uqwc = get_qwc_unitary(comm_groups[1])
print('This is unitary, U * U^+ = I ')
print(uqwc * uqwc)

This is unitary, U * U^+ = I 
(0.9999999999999996+0j) []


Applying this unitary gives the qubit-wise commuting form of the first mutually commuting group

In [12]:
qwc = remove_complex(uqwc * comm_groups[1] * uqwc)
print(qwc)

-0.32760818967480876 [] +
0.15542669077992827 [X0] +
0.15660062488237952 [X0 X1] +
0.04919764587136755 [X0 X1 Z3] +
0.10622904490856076 [X0 X2] +
-0.04919764587136755 [X0 Z3] +
0.10622904490856076 [X1] +
0.15542669077992827 [X1 X2] +
-0.04919764587136755 [X1 X2 Z3] +
0.16326768673564335 [X2] +
0.04919764587136755 [X2 Z3]


In addition, current quantum computer can measure only the $z$ operators. Thus, QWC fragments with $x$ or $y$ operators
require extra single-qubit unitaries that rotate them into $z$.

In [18]:
uz = get_zform_unitary(qwc)
print("Checking whether U * U^+ is identity: {}".format(uz * uz))

allz = remove_complex(uz * qwc * uz)
print("\nThe all-z form of qwc fragment:\n{}".format(allz))

Checking whether U * U^+ is identity: 0.9999999999999998 []

The all-z form of qwc fragment:
-0.3276081896748086 [] +
0.15542669077992818 [Z0] +
0.15660062488237944 [Z0 Z1] +
0.04919764587136754 [Z0 Z1 Z3] +
0.1062290449085607 [Z0 Z2] +
-0.04919764587136754 [Z0 Z3] +
0.1062290449085607 [Z1] +
0.15542669077992818 [Z1 Z2] +
-0.04919764587136754 [Z1 Z2 Z3] +
0.16326768673564326 [Z2] +
0.04919764587136754 [Z2 Z3]


Now we will do it for the molecules in question:$H_4$, $N_2$, and $H_2O$.

In [45]:
def return_qubit_ham(mol: str, geometry: float, basis: str, transf: str = 'jw', active: dict = None):
    xyz = get_molecular_data(mol, geometry, True)
    molecule = tq.quantumchemistry.Molecule(geometry=xyz, basis_set=basis,
                                            transformation=transf, active_orbitals=active)
    ham = molecule.make_hamiltonian()
    ham = ham.to_openfermion()
    return ham

def return_measurement_fragments(ham):
    comm_groups = get_commuting_group(ham)
    out_dict = {}
    for key, group in comm_groups.items():
        qwc_unitary = get_qwc_unitary(group)
        qwc_op = remove_complex(qwc_unitary * group * qwc_unitary)
        unitary_z = get_zform_unitary(qwc_op)
        z_measurements = remove_complex(unitary_z * qwc_op * unitary_z)
        out_dict[key] = z_measurements
    return out_dict

In [39]:
#H_4
angles = np.linspace(85, 95, 10)
h4_dict = {}
for a in angles:
    ham = return_qubit_ham('h4', a, 'sto-3g', 'jw')
    h4_dict[a] = return_measurement_fragments(ham)

In [43]:
# generating this takes a long time - let's save so we have it available
with open("../../week2_data/h4_jw_measurements.pkl", 'wb') as f:
    pkl.dump(h4_dict, f)

In [46]:
#H2O
basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
lengths = np.linspace(0.5, 1, 10)
h2o_dict = {}
for l in lengths:
    ham = return_qubit_ham('h2o', l, basis, 'jw', active)
    h2o_dict[l] = return_measurement_fragments(ham)

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed 

In [47]:
with open("../../week2_data/h2o_jw_measurements.pkl", 'wb') as f:
    pkl.dump(h2o_dict, f)

In [50]:
# N2 WARNING: Loooong
basis = '6-31g'
active = {'AG': [1, 3], 'B1U': [0, 1], 'B3U': [0]}
lengths = np.linspace(0.1, 2, 10)
n2_dict = {}
for l in lengths:
    ham = return_qubit_ham('n2', l, basis, 'jw', active)
    n2_dict[l] = return_measurement_fragments(ham)
    with open("../../week2_data/n2_jw_measurements.pkl", 'wb') as f:
        pkl.dump(n2_dict, f)

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed 